<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Germán Urrea
- Nombre de alumno 2: - 


### **Link de repositorio de GitHub:** `https://github.com/G-Urrea/labs_mds_german_urrea`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 88.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gdown
try:
  df_players = pd.read_csv('stats_players.csv')
except:
  url = "https://drive.google.com/uc?id=1j1ir0YF68akv5z4ikKfySt6AQaU_4QAH"
  gdown.download(url, 'stats_players.csv')
  df_players = pd.read_csv('stats_players.csv')
df_players

Downloading...
From: https://drive.google.com/uc?id=1j1ir0YF68akv5z4ikKfySt6AQaU_4QAH
To: /content/stats_players.csv
100%|██████████| 2.58M/2.58M [00:00<00:00, 114MB/s]


,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

Se crea una nueva columna "label", en donde un valor 1 indica un jugador seleccionado, mienntras que 0 indica un jugador no seleccionado

In [4]:
df_players['label'] = df_players["National_Position"].isna().apply(lambda x: 0 if x else 1)

Acto seguido se cuenta el número de clases

In [5]:
df_players['label'].value_counts()/len(df_players['label'])

0    0.938879
1    0.061121
Name: label, dtype: float64

La mayoría de jugadores no son seleccionados, alrededor del $93\%$, lo que quiere decir que estamos ante un caso de clasificación altamente desbalanceado.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

Para el preprocesamiento se toman las siguientes decisiones:
- Deshacerse de la columna `National_Position`, dado que las etiquetas se derivan de esta y puede producirse data leakage.
- Deshacerse de la columna `Name`, pues para que el clasificador sea más generalizable, el nombre no debería ser un factor relevante a la hora de definir si un jugador será seleccionado.
- Aplicar One-Hot Encoding en las caracteristicas categoricas, para poder aplicar estas en modelos matematicos. Se escoge One-Hot Encoding principalmente porque de esta forma se evita representar en una escala ordenada las caracteristicas, las que en este caso no se adaptarían bien  ya que, por ejemplo, la caracteristica `Work_Rate` posee los valores "High / Medium" y "Medium/High", los que no serían evidentes de traducir a una escala numérica convencional.
- Aplicar un **StandardScaler** a las características numéricas. Esto pues algunas de estas presentan comportamientos similares a una distribución normal, como las variables `Strength` y `Composure`. De esta forma se llevaría a todas las variables a un rango comparable, beneficiandose principalmente aquellas que siguen una distribución similar a una normal.

In [6]:
categoric_columns = df_players.dtypes[df_players.dtypes=='object']
categoric_columns

Name                 object
Nationality          object
National_Position    object
Club_Position        object
Preffered_Foot       object
Work_Rate            object
dtype: object

In [7]:
numeric_columns = df_players.dtypes[df_players.dtypes =='int64']
numeric_columns = [x for x in numeric_columns.index if x!='label']

In [8]:
transformer_players = ColumnTransformer([
    ('Encoding', OneHotEncoder(handle_unknown='ignore'), ['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate']),
     ('standarization', StandardScaler(), numeric_columns)
     ],
     remainder='drop'
    )

In [9]:
def pipeline_0(classifier):
  pipe = Pipeline([
      ('transformer', transformer_players),
      ('classifier', classifier)
  ])
  return pipe

Siguiendo los lineamientos de Scikit-Learn, se ocuparán los siguientes clasificadores:
- **Linear SVC**: Dado que se está ante un problema de clasificación con datos etiquetados y menos de 100k muestras.
- **KNN**: Dado que es el clasificador recomendado cuando la clasificación no funciona y se tiene caracteristifcas numericas.
- **Random Forest**: Pues se aconseja el uso de clasificadores ensemble en caso de que KNN no sea suficiente, y Random Forest es una de las opciones clásicas para este tipo de clasificadores.

In [10]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [11]:
pipe_svc = pipeline_0(SVC(kernel='linear', random_state=42))
pipe_knn = pipeline_0(KNeighborsClassifier())
pipe_rf = pipeline_0(RandomForestClassifier(random_state=42))

In [12]:
pipe_rf

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Encoding',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('standarization',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve', ...])])),
                ('classifier', RandomForestClassifier(random_state=42))])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

Se escoge una proporción estándarde train/test, en este caso $7:3$.

In [13]:
X = df_players.drop(columns = ['label'])
y = df_players['label']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify = y)

In [15]:
def proporcion(labels):
  '''
  Recibe labels binarios (0,1).
  Entrega porcentaje de clase positiva
  '''
  return labels.sum()/len(labels)

print(f"Proporción original: {proporcion(y)}")
print(f"Proporción train: {proporcion(y_train)}")
print(f"Proporción train: {proporcion(y_test)}")

Proporción original: 0.061121219012963385
Proporción train: 0.061083583786857285
Proporción train: 0.061209020276672355


In [16]:
pipe_svc.fit(x_train, y_train)
pipe_knn.fit(x_train, y_train)
pipe_rf.fit(x_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Encoding',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('standarization',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve', ...])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [17]:
y_pred_svc = pipe_svc.predict(x_test) 
y_pred_knn = pipe_knn.predict(x_test) 
y_pred_rf = pipe_rf.predict(x_test) 

**Respuesta**:



In [18]:
print("Resultados SVC Linear")
print(classification_report(y_test, y_pred_svc))

Resultados SVC Linear
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4954
           1       0.64      0.18      0.28       323

    accuracy                           0.94      5277
   macro avg       0.80      0.59      0.63      5277
weighted avg       0.93      0.94      0.93      5277



In [19]:
print("Resultados KNN")
print(classification_report(y_test, y_pred_knn))

Resultados KNN
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4954
           1       0.40      0.05      0.09       323

    accuracy                           0.94      5277
   macro avg       0.67      0.52      0.53      5277
weighted avg       0.91      0.94      0.91      5277



In [20]:
print("Resultados Random Forest")
print(classification_report(y_test, y_pred_rf))

Resultados Random Forest
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.66      0.08      0.15       323

    accuracy                           0.94      5277
   macro avg       0.80      0.54      0.56      5277
weighted avg       0.93      0.94      0.92      5277



- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

Los 3 clasificadores dejan bastante que desear en las prediciones de seleccionados.
- Para la métrica **Recall**, que indica el porcentaje de ejemplos acertados correctamente entre los ejemplos que de verdad pertenecen a la clase, osea, el rendimiento respecto a los datos, ninguno de los clasificadores alcanza más de un $0.16$. Esto quiere decir que, respecto al conjunto de datos de prueba, ninguno de los clasificadores logra acertar a una canntidad significativa de ejemplos.
- Para la métrica **Precision**, que indica el porcentaje de ejemplos acertados correctamente entre las predicciones para dicha clase, osea, el rendimiento respecto al clasificador, el rendimiento es relativamente mejor. En este caso el clasificador **KNN** es el que peor se desempeña, con $0.4$ de precisión, mientras que **Random Forest** es el mejor, con un $0.7$ de precisión. Esto quiere decir que, en general , los clasificadores rinden relativamente mejor si los medimos respecto a sus propias predicciones de la clase positiva.
- Para la métrica **Accuracy**, que indica el porcentaje general de predicciones correctas (tanto positivas como negativas), los resultados de los 3 clasificadores son bastante mejorables, pues no suelen ser más altas que $0.61$. 
- Por otro lado, para el *weighted average*, que toma en cuenta el desbalance de clases en la ponderación del score **F1**, el cual a su vez es una ponderación de precision y recall, los resultados son bastante altos, generalmente superando el $0.9$.

En este sentido es claro que el desbalance de clases afecta la clasificación, pues los clasificadores rinden bastante mejor respecto a la clase negativa, y esto no es de extrañar, pues el $93\%$ de las muestras pertenecen a esta clase.

Los resultados son mejorables, en primer lugar habría que tratar con el desbalance de clases. Para tratar con el desbalance, se pueden recolectar más datos de la clase positiva, sin embargo en este caso sería más directo aplicar alguna técnica de *undersampling* u *oversampling*, para minimizar el impacto del desbalance y mejorar las métricas. Posterior a esto también se podrían probar otras técnicas, como ajuste de hiperparámetros mediante validación cruzada, con el objetivo de refinar los resultados respecto a los modelos utilizados. 

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [21]:
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

mapper = {
    'ataque':ataque,
    'central_ataque':central_ataque,
    "central": central,
    "central_defensa": central_defensa,
    "defensa": defensa,
    "arquero": arquero
}

position_mapper = {}

for positions in mapper:
  for pos in mapper[positions]:
    # guardar mapeo (ej: ST-> ataque)
    position_mapper[pos] = positions


In [22]:
df_players['position_label'] = df_players['Club_Position'].apply(lambda x: position_mapper.get(x))

Se elimina cualquier muestra que no corresponda a las etiquetas definidas, pues no aportarían a la clasificación.

In [23]:
# Botar filas sin label apropiado
df_positions = df_players.dropna(subset=['position_label'])
print(len(df_positions))

3939


In [24]:
print(df_positions['position_label'].value_counts())
print()
print(df_positions['position_label'].value_counts()/len(df_positions['position_label']))

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: position_label, dtype: int64

defensa            0.299568
central            0.230261
arquero            0.160447
central_ataque     0.147499
ataque             0.109165
central_defensa    0.053059
Name: position_label, dtype: float64


En este caso el número de muestras es mucho menor que antes, debido a la eliminación de muestras sin una etiqueta válida. En este caso el equilibrio de clases es mejor que antes, aunque sigue existiendo cierto desbalance, en este caso la clase más favorecida es "defensa", mientras que se tienen muy pocas muestras de "central_defensa", alrededor del $5\%$ de las muestras, por lo que puede que existan problemas al predecir dicha clase.

In [25]:
df_positions['National_Position'].isna().sum()

3573

### Definición de pipeline

Respecto a la transformación de variables, el único cambio significativo será eliminar la variable `Club_Position` como caracteristica, pues traería consigo "data leakages". A su vez, se utilizarán las etiquetas creadas en los pasos anteriores, osea, si el jugador es seleccionado nacional o no, y en cambio no se utilizará la caracteristica `National_Position`, principalmente por la existencia de nans.

In [26]:
transformer_positions = ColumnTransformer([
    ('Encoding', OneHotEncoder(handle_unknown='ignore'), ['Nationality', 'Preffered_Foot', 'Work_Rate']),
     ('standarization', StandardScaler(), numeric_columns),# Mismas columnas numéricas definidas antes
     ('pass', 'passthrough', ['label'])
     ],
     remainder='drop'
    )

Respecto a los clasificadores a utilizar, estos no cambiarán, debido a que se sigue trabajando con caracteristicas numéricas y con una cantidad de datos relativamente baja.

In [27]:
def pipeline_1(clf):
  return Pipeline([
      ('transformer', transformer_positions),
      ('classifier', clf)
  ])

In [28]:
pipe_svc_1 = pipeline_1(SVC(kernel='linear', random_state=42))
pipe_knn_1 = pipeline_1(KNeighborsClassifier())
pipe_rf_1 = pipeline_1(RandomForestClassifier(random_state=42))

In [29]:
x,y = df_positions.drop(columns=['position_label']), df_positions['position_label']

In [30]:
# Es necesario un nuevo split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42, stratify = y)

In [31]:
pipe_svc_1.fit(x_train, y_train)
pipe_knn_1.fit(x_train, y_train)
pipe_rf_1.fit(x_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Encoding',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('standarization',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve', ...]),
                                                 ('pass', 'passthrough',
                                                  ['label'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

### Clasificación

In [32]:
y_pred_svc_1 = pipe_svc_1.predict(x_test)
y_pred_knn_1 = pipe_knn_1.predict(x_test)
y_pred_rf_1 = pipe_rf_1.predict(x_test)

In [33]:
print("Resultados SVC Linear")
print(classification_report(y_test, y_pred_svc_1))

Resultados SVC Linear
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.76      0.81      0.78       129
        central       0.56      0.60      0.58       272
 central_ataque       0.47      0.37      0.41       174
central_defensa       0.34      0.16      0.22        63
        defensa       0.84      0.94      0.89       354

       accuracy                           0.73      1182
      macro avg       0.66      0.65      0.65      1182
   weighted avg       0.71      0.73      0.72      1182



In [34]:
print("Resultados KNN")
print(classification_report(y_test,y_pred_knn_1))

Resultados KNN
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.68      0.84      0.75       129
        central       0.54      0.57      0.56       272
 central_ataque       0.45      0.28      0.35       174
central_defensa       0.33      0.14      0.20        63
        defensa       0.81      0.94      0.87       354

       accuracy                           0.71      1182
      macro avg       0.63      0.63      0.62      1182
   weighted avg       0.68      0.71      0.69      1182



In [109]:
print("Resultados Random Forest")
print(classification_report(y_test,y_pred_rf_1))

Resultados Random Forest
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.75      0.82      0.79       129
        central       0.55      0.62      0.58       272
 central_ataque       0.47      0.29      0.36       174
central_defensa       0.47      0.11      0.18        63
        defensa       0.80      0.96      0.87       354

       accuracy                           0.73      1182
      macro avg       0.67      0.63      0.63      1182
   weighted avg       0.70      0.73      0.70      1182



En este caso los resultados de clasificación son mixtos,  teniendo los 3 clasificadores rendimientos muy similares (alrededor de $0.7$ de accuracy).
-  En general, los clasificadores son capaces de distinguir a la perfección a los arqueros, además de obtener resultados decentes para "ataque" y "defensa". En este caso resaltan los resultados para la clase "ataque", tomando en cuenta que la cantidad de muestras es relativamente baja (alrededor del $10\%$) en comparación al resto de clases.
- La clase "Central" en general presenta métricas bajas, pues no están muy lejos de lo que se esperaría de una predicción al azar. En este caso tanto *Precision* como *Recall* están en valores similares, lo que puede indicar que se realiza una cantidad de predicciones parecida al numéro real de ejemplos de la clase.
- Por otro lado, las clases "central_ataque" y "central_defensa" tienen métricas bastante mejorables. En este sentido resalta que ambas tienen *Precision* más alto que *Recall*, lo que lleva a pensar que, en general, se realizan pocas predicciones de estas clases, lo que explicaría el *Recall* tan bajo.

En este caso, para mejorar los resultados se podría seguir una metodología similar a la sugerida para la mejora de la predicción de seleccionados. Sin embargo, en este caso tal vez sea necesario realizar una exploración de datos más profunda para definir un curso de acción, pues por ejemplo la clase "central" es la que posee la segunda cantidad de muestras más grande, y sin embargo no se obtienen buenos resultados en sus predicciones.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [36]:
### Código aquí ###
try:
  df_sueldos = pd.read_csv('sueldos.csv')
except:
  url = "https://drive.google.com/uc?id=1R2HIRG7UFxKJHTKF-jDJD7jef0I_FXoY"
  gdown.download(url, 'sueldos.csv')
  df_sueldos = pd.read_csv('sueldos.csv')


Downloading...
From: https://drive.google.com/uc?id=1R2HIRG7UFxKJHTKF-jDJD7jef0I_FXoY
To: /content/sueldos.csv
100%|██████████| 68.8k/68.8k [00:00<00:00, 43.4MB/s]


In [44]:
df_sueldos.rename(columns ={'Player':'Name'}, inplace=True)
df_sueldos.head()

,Unnamed: 0,Name,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0


Se aplica un inner join con los datos ya existentes para de esta forma utilizar las caracteristicas ya conocidas.

In [47]:
df_sueldos = df_sueldos.merge(df_players, on='Name')

Al igual que en la clasificación, se aplicarán transformaciones de estandarización y encoding a las caracteristicas. Respecto a las etiquetas generadas con anterioridad, se utilizará la etiqueta que señala si un jugador es seleccionado o no (`label`), mientras que, para no perder información por valores nulos, se utilizará la característica `Club_Position` en lugar de la etiqueta derivada de esta.

In [46]:
transformer_salary = ColumnTransformer([
    ('Encoding', OneHotEncoder(handle_unknown='ignore'), ['Club_Position', 'Nationality', 'Preffered_Foot', 'Work_Rate']),
     ('standarization', StandardScaler(), numeric_columns),# Mismas columnas numéricas definidas antes
     ('pass', 'passthrough', ['label'])
     ],
     remainder='drop'
    )

In [58]:
print(df_sueldos.shape)

(1866, 43)


En este caso, la cantidad de muestras no es precisamente alta, pues se tienen alrededor de 1800, por otro lado, la cantidad de caracteristicas es de alrededor de 40, y se espera que con el encoding el número aumente entre 10 y 20 caracteristicas más. Dado que la cantidad de caracteristicas es considerable, pero no se tiene certeza de la importancia de cada una en la tarea de regresión, se seguirán los lineamientos de la guía de "sklearn" para cada caso, osea, se escogerá un regresor para el caso en que sólo unas cuantas caracteristicas importan, y uno para el caso contrario. En este contexto, se trabajará con las regresiones Lasso y Ridge, para así tener un punto de comparación entre las penalizaciones L1 y L2.

In [59]:
def pipeline_regr(model):
  return Pipeline(
      [
          ('transformer', transformer_salary),
          ('regressor', model)
      ]
  )

In [77]:
x, y = df_sueldos.drop(columns=['Weekly Salary']), df_sueldos['Weekly Salary']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, train_size = 0.7)

In [78]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

pipe_lasso = pipeline_regr(Lasso())
pipe_ridge = pipeline_regr(Ridge())

In [79]:
pipe_lasso.fit(x_train, y_train)
pipe_ridge.fit(x_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Encoding',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Club_Position',
                                                   'Nationality',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('standarization',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve', ...]),
                                                 ('pass', 'passthrough',
                                                  ['label'])])),
                ('regressor', Ridge())])

**Respuesta**:


Para la evaluación de los regresores se utiliza la métrica $R^2$, también conocida como coeficiente de determinación, esta es una métrica definida en el rango $[-∞,1]$ que mide la proporción de varianza de las predicciones respecto a los verdaderos valores. En este caso el mejor valor posible es 1, indicando predicciones perfectas, mientras que valores cercanos a 0 y negativos indican una regresión deficiente.

In [80]:
r2_lasso = r2_score(y_test, pipe_lasso.predict(x_test))
r2_ridge = r2_score(y_test, pipe_ridge.predict(x_test))

In [81]:
print(f"LASSO: {r2_lasso}")
print(f"Ridge: {r2_ridge}")

LASSO: -0.07574850718499815
Ridge: 0.009775650777637757


En primera instancia los regresores no obtienen buenos resultados con el conjunto de prueba, pues el $R^2$ es más cercano a 0 que a 1 en ambos casos, de hecho LASO obtiene un $R^2$ negativo, lo que es un indicio de un pésimo ajuste. En este caso se siguen los lineamientos de *sklearn* cuando no funciona una regresión Ridge, que es intentar con un regresor de tipo *ensemble*. En este caso se probará con un *Random Forest Regressor*



In [82]:
from sklearn.ensemble import RandomForestRegressor

In [83]:
pipe_rfr = pipeline_regr(RandomForestRegressor())
pipe_rfr.fit(x_train, y_train)
r2_rfr = r2_score(y_test, pipe_rfr.predict(x_test))
print(f"Random Forest Regressor: {r2_rfr}")

Random Forest Regressor: 0.40650691455977717


Los resultados obtenidos con el regresor Random Forest son mucho mejores, superando el umbral establecido de $0.35$.

En general, los resultados dejaron que desear, pues los primeros 2 modelos utilizados obtuvieron un $R^2$ altamente deficiente. En este caso, el uso de modelos *ensemble* fue beneficioso, pues se alcanzaron resultados mejores. Se puede teorizar que la ventaja de Random Forest radica en que este captura mejor relaciones no lineales, mientras que los otros 2 modelos son de regresión lineal.

De cualquier forma, es claro que se necesita un mayor trabajo en este problema para mejorar las métricas de los modelos. En este caso el método más directo sería probar otros tipos de regresores no lineales e ir ajustando los hiperparámetros de estos para intentar mejorar los resultados. Sin embargo, una mayor cantidad de muestras podría ayudar a capturar comportamientos que actualmente los modelos no sean capaces de identificar por falta de evidencia. Por otro lado, tal vez la cantidad de dinero que gana un futbolista tenga que ver más con aspectos de marketing, como su popularidad en redes sociales y las campañas de publicidad en las que ha aparecido, por lo que también podría ser de utilidad poseer este tipo de características.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>